## Post-Hoc ANOVA T-Tests (JATA v4)

*Last update: 2022-05-24*

In [2]:
import pandas as pd
import numpy as np
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import importlib
from scipy.stats import circvar, circstd, pearsonr
from pingouin import pairwise_ttests, pairwise_tests
import pingouin as pg


from trial import Trial
from util import dist
from constants import COND_LABELS, FS
import plots
import analysis_helper

importlib.reload(plots)
importlib.reload(analysis_helper)

plt.style.use('./jg.mplstyle')

### Load data

In [3]:
df_trials = pd.read_pickle('../data/df_trials_v4_spatial.pickle')

In [4]:
DVs = ['fairness', 'goal_color_split', 'successful', 'total_moves']

### Drop dyads with incomplete trial counts

In [5]:
# Drop practice
df_trials = df_trials[df_trials.practice == 0]

In [6]:
df_trials.columns

Index(['p0', 'ave_gem_diag', 'avg_vel', 'cond_bal', 'cond_count', 'cond_goals',
       'duration', 'dyad_med_move_wait', 'end_type', 'fairness',
       'first_counter_block', 'first_move_player', 'first_mover_time',
       'goal_color_split', 'goal_pct_diff', 'map', 'match_id', 'match_nick',
       'mean_dist', 'move_balance', 'occupancy_p0', 'occupancy_p1',
       'p0_avg_vel', 'p0_c0_count', 'p0_c0_goal_pct', 'p0_c1_count',
       'p0_c1_goal_pct', 'p0_first_move_dir', 'p0_first_move_time',
       'p0_gem_diag', 'p0_goal_count', 'p0_goal_pct', 'p0_med_move_wait',
       'p0_n_moves', 'p1', 'p1_avg_vel', 'p1_c0_count', 'p1_c1_count',
       'p1_first_move_dir', 'p1_first_move_time', 'p1_gem_diag',
       'p1_goal_count', 'p1_goal_pct', 'p1_med_move_wait', 'p1_n_moves',
       'practice', 'recent_ct_fairness', 'recent_ct_goal_color_split', 'score',
       'second_mover_time', 'spatial_cons', 'successful', 'theta_diff',
       'theta_mean_p0', 'theta_mean_p1', 'total_moves', 'trial', 't

### Prepare data in long format for ttests

In [7]:
match_nicks = df_trials.match_nick.unique()
count_levels = ["C", "G"]
bal_levels = ['E', 'FB']
dv_grouped = df_trials.groupby(['match_nick', 'cond_count', 'cond_bal'])[DVs].mean()
data = {}  # match_nick -> data
series_rows = []
for key, row in dv_grouped.iterrows():
    match_nick, count, bal = key
    for count in count_levels:
        for bal in bal_levels:
            first_counter_block = df_trials[df_trials.match_nick == match_nick].iloc[0].first_counter_block
            series_row = {
                'match_nick': match_nick,
                'counter': count,
                'balance': bal,
                'first_counter_block': 'g' + first_counter_block,
                'block': 1 if first_counter_block == count else 2
            }
            for dv in DVs:
                series_row[dv] = dv_grouped.loc[match_nick, count, bal][dv]
            series_rows.append(series_row)
            
df_long = pd.DataFrame(series_rows)
df_long

,match_nick,counter,balance,first_counter_block,block,fairness,goal_color_split,successful,total_moves
0,17g7hb[CG],C,E,gC,1,0.430417,0.631667,0.90,47.05
1,17g7hb[CG],C,FB,gC,1,0.359615,0.925833,0.90,49.55
2,17g7hb[CG],G,E,gC,2,0.175000,0.833333,0.95,47.80
3,17g7hb[CG],G,FB,gC,2,0.234722,1.000000,0.95,54.15
4,17g7hb[CG],C,E,gC,1,0.430417,0.631667,0.90,47.05
...,...,...,...,...,...,...,...,...,...
379,zkcyp1[GC],G,FB,gG,1,0.287401,0.925833,0.85,49.25
380,zkcyp1[GC],C,E,gG,2,0.483333,0.992857,0.95,52.80
381,zkcyp1[GC],C,FB,gG,2,0.451389,0.555833,0.95,54.25
382,zkcyp1[GC],G,E,gG,1,0.485203,1.000000,0.90,54.60


In [8]:
df_long.groupby(['block', 'first_counter_block'])['goal_color_split'].mean()

block  first_counter_block
1      gC                     0.772729
       gG                     0.878962
2      gC                     0.897353
       gG                     0.852842
Name: goal_color_split, dtype: float64

In [11]:
dv = DVs[1]
print(dv)
df_b1 = df_long[df_long.block == 1]
posthoc = pairwise_tests(data=df_b1, dv=dv, subject='match_nick',
                between='counter', 
                within_first=False,
                interaction=False,
                effsize='eta-square')
posthoc


goal_color_split


/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,eta-square
0,counter,C,G,False,True,-6.109147,190.0,two-sided,5.548807e-09,1.79e+06,0.162748


In [12]:

dv = DVs[1]
print(dv)
df_b2 = df_long[df_long.block == 2]
posthoc = pairwise_tests(data=df_b2, dv=dv, subject='match_nick',
                between='counter', 
                within_first=False,
                interaction=False,
                effsize='eta-square')
posthoc

goal_color_split


/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,eta-square
0,counter,C,G,False,True,-2.318893,190.0,two-sided,0.021465,1.89,0.027244


In [10]:
dv = DVs[1]
print(dv)
posthoc = pairwise_ttests(data=df_long, dv=dv, within='counter', subject='match_nick',
                between='first_counter_block', 
                within_first=False,
                interaction=True, padjust='bonf',
                effsize='eta-square')
posthoc

goal_color_split


/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:477: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = data.groupby([subject, f], as_index=False, observed=True, sort=True).mean()
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in

,Contrast,first_counter_block,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,eta-square
0,first_counter_block,-,gC,gG,False,True,-0.803901,22.0,two-sided,0.430056,NaN,NaN,0.472,0.026221
1,counter,-,C,G,True,True,-3.333193,23.0,two-sided,0.002890,NaN,NaN,13.939,0.107638
2,first_counter_block * counter,gC,C,G,True,True,-3.585066,11.0,two-sided,0.004280,0.008561,bonf,11.862,0.322591
3,first_counter_block * counter,gG,C,G,True,True,-1.181385,11.0,two-sided,0.262362,0.524723,bonf,0.51,0.011530


In [11]:
dv = DVs[2]
print(dv)
posthoc = pairwise_ttests(data=df_long, dv=dv, within=['counter', 'balance'], subject='match_nick',
                between='first_counter_block', 
                within_first=False,
                interaction=True, padjust='bonf',
                effsize='cohen')
posthoc

successful


/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:477: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = data.groupby([subject, f], as_index=False, observed=True, sort=True).mean()
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:477: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = data.gro

,Contrast,counter,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,counter,-,C,G,True,True,1.746283,23.0,two-sided,0.094106,NaN,NaN,0.799,0.316382
1,balance,-,E,FB,True,True,-0.410586,23.0,two-sided,0.685176,NaN,NaN,0.232,-0.034345
2,counter * balance,C,E,FB,True,True,-0.465057,23.0,two-sided,0.646264,1.0,bonf,0.237,-0.040178
3,counter * balance,G,E,FB,True,True,-0.233181,23.0,two-sided,0.817683,1.0,bonf,0.22,-0.023768


In [12]:
dv = DVs[3]
print(dv)
posthoc = pairwise_ttests(data=df_long, dv=dv, within='counter', subject='match_nick',
                between='first_counter_block', 
                within_first=False,
                interaction=True, padjust='bonf',
                effsize='eta-square')
posthoc

total_moves


/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:27: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/pingouin/pairwise.py:477: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = data.groupby([subject, f], as_index=False, observed=True, sort=True).mean()
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: overflow encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/opt/miniconda3/envs/jointaction/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: overflow encountered in

,Contrast,first_counter_block,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,eta-square
0,first_counter_block,-,gC,gG,False,True,-0.985893,22.0,two-sided,0.334905,NaN,NaN,0.531,0.038923
1,counter,-,C,G,True,True,3.097258,23.0,two-sided,0.005080,NaN,NaN,8.59,0.036453
2,first_counter_block * counter,gC,C,G,True,True,2.347558,11.0,two-sided,0.038653,0.077307,bonf,2.021,0.035200
3,first_counter_block * counter,gG,C,G,True,True,1.939775,11.0,two-sided,0.078472,0.156943,bonf,1.181,0.040419
